In [1]:
# imports and utils

import tensorflow.compat.v2 as tf
import ddsp.training
_AUTOTUNE = tf.data.experimental.AUTOTUNE
from IPython.display import Audio, display
from livelossplot import PlotLosses
import numpy as np
import random 
import matplotlib.pyplot as plt
import tensorflow_datasets as tfds
import time
import data
import random
import copy
import pydash
import tqdm
import soundfile
import os
import model

# define constants
CLIP_S=4
SAMPLE_RATE=48000
N_SAMPLES=SAMPLE_RATE*CLIP_S
SEED=1
FT_FRAME_RATE=250

tf.random.set_seed(
    SEED
)
np.random.seed(SEED)
random.seed(SEED)

# define some utilis
def play(audio):
  display(Audio(audio,rate=SAMPLE_RATE))


In [2]:
USE_NSYNTH=False
INSTRUMENT_FAMILY="Trombone"

In [3]:
IR_DURATION=1
Z_SIZE=1024 if INSTRUMENT_FAMILY=="**" else 512
N_INSTRUMENTS=200
BIDIRECTIONAL=True
USE_F0_CONFIDENCE=True
N_NOISE_MAGNITUDES=192
N_HARMONICS=192
                 
ae=model.get_model(SAMPLE_RATE,CLIP_S,FT_FRAME_RATE,Z_SIZE,N_INSTRUMENTS,IR_DURATION,BIDIRECTIONAL,USE_F0_CONFIDENCE,N_HARMONICS,N_NOISE_MAGNITUDES)

# define loss
fft_sizes = [64]
while fft_sizes[-1]<SAMPLE_RATE//4:
    fft_sizes.append(fft_sizes[-1]*2)

spectral_loss = ddsp.losses.SpectralLoss(loss_type='L1',
                                            fft_sizes=fft_sizes,
                                            mag_weight=1.0,
                                            logmag_weight=1.0)

ResourceExhaustedError: OOM when allocating tensor with shape[200,48000] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc [Op:Fill]

In [ ]:


if USE_NSYNTH:
    tfds.load("nsynth/gansynth_subset.f0_and_loudness",split="train", try_gcs=False,download=True) 
    trn_data_provider = data.CustomNSynthTfds(data_dir="/root/tensorflow_datasets/",split="train")
    tfds.load("nsynth/gansynth_subset.f0_and_loudness",split="valid", try_gcs=False,download=True) 
    val_data_provider = data.CustomNSynthTfds(data_dir="/root/tensorflow_datasets/",split="valid")
    def crepe_is_certain(x):
        is_playing = tf.cast(x["loudness_db"]>-100.0,dtype=tf.float32)
        average_certainty=tf.reduce_sum(x["f0_confidence"]*is_playing)/tf.reduce_sum(is_playing)
        return average_certainty
    def preprocess_dataset(dataset):
        if INSTRUMENT_FAMILY!="all":
            dataset=dataset.filter(lambda x: x["instrument_family"]==INSTRUMENT_FAMILY)
        return dataset
    trn_dataset = preprocess_dataset(trn_data_provider.get_dataset())
    val_dataset = preprocess_dataset(val_data_provider.get_dataset())

else:

    
    trn_data_provider=data.MultiTFRecordProvider(f"datasets/AIR/tfr48k/dev/{INSTRUMENT_FAMILY}/*",sample_rate=SAMPLE_RATE)
    trn_dataset= trn_data_provider.get_dataset()
    
    val_data_provider=data.MultiTFRecordProvider(f"datasets/AIR/tfr48k/tst/{INSTRUMENT_FAMILY}/*",sample_rate=SAMPLE_RATE)
    val_dataset=val_data_provider.get_dataset(shuffle=False)

In [ ]:
# remove some samples if number of recordings greater than model capacity
trn_dataset = trn_dataset.filter(lambda x: int(x["instrument_idx"])<N_INSTRUMENTS)

In [ ]:
checkpoint_path=f"checkpoints/48k_{'bidir' if BIDIRECTIONAL else 'unidir'}_z{Z_SIZE}_conv_family_{INSTRUMENT_FAMILY}{'_f0c' if USE_F0_CONFIDENCE else ''}"

print(checkpoint_path)
try:
    print("loading checkpoint")
    ae.load_weights(checkpoint_path)
except:
    print("couldn't load checkpoint")
    pass

In [ ]:
plotlosses = PlotLosses()

In [ ]:
## training loop with adam

BATCH_SIZE=1
batched_trn_dataset=trn_dataset.shuffle(10000).batch(BATCH_SIZE,drop_remainder=True)

# 1e-4 was good for saxophone (got us to 4.7-ish in 20 hours our so)
optimizer = tf.keras.optimizers.Adam(learning_rate=2e-5)

e=0
while True:
  batch_counter=0
  epoch_loss=0   
  for batch in batched_trn_dataset:
    with tf.GradientTape() as tape:
        output=ae(batch,train_shared=True)
        loss_value=spectral_loss(batch["audio"],output["audio_synth"])
        gradients = tape.gradient(loss_value, ae.trainable_variables)
        epoch_loss+=loss_value.numpy()
        optimizer.apply_gradients(zip(gradients, ae.trainable_variables))
        
    if batch_counter % 10==0:
        print(f"batch nr {batch_counter}, loss: {loss_value.numpy()}")

    if batch_counter ==0:    
        play(tf.reshape(output["audio"],(-1)))
        play(tf.reshape(output['audio_synth'],(-1)))
        play(tf.reshape(output['harmonic+fn']["signal"],(-1))) 
        play(tf.reshape(output['harmonic']["signal"],(-1))) 
        play(tf.reshape(output["fn"]["signal"],(-1)))
        play(tf.reshape(output["ir"],(-1)))
        
    if batch_counter>1000-1 and batch_counter % 1000==0:
        ae.save_weights(checkpoint_path)

    batch_counter+=1
    
  plotlosses.update({'loss': epoch_loss/batch_counter,})
  plotlosses.send()
  ae.save_weights(checkpoint_path)
  e+=1

In [ ]:
# define some data utilities

N_FIT_SECONDS = 16
    
TRAIN_SHARED=False

USE_FNR=True

EARLY_REFLECTION_DURATION=0.2

def join_batch(batch):
    for key in batch.keys():
        assert len(batch[key].shape)<3
        if len(batch[key].shape)==2:
            batch[key]=tf.reshape(batch[key],(1,-1))
    return batch

def window_signal(a,window_len,hop_len):
     assert(a.shape[0]==1)
     windows=[]
     start_frame=0
     while True:
        windows.append(a[:,start_frame:start_frame+window_len,...])
        start_frame+=hop_len
        if start_frame > a.shape[1]-window_len:
            break
     return tf.concat(windows,axis=0)

def window_sample(instance,win_s,hop_s):
    instance["audio"]=window_signal(instance["audio"],win_s*SAMPLE_RATE,hop_s*SAMPLE_RATE)
    for key in ["f0_hz","loudness_db","f0_confidence"]:
        instance[key]=window_signal(instance[key],win_s*FT_FRAME_RATE,hop_s*FT_FRAME_RATE)
    instance["instrument"]=tf.repeat(instance["instrument"][0],(instance["audio"].shape[0]))
    instance["instrument_idx"]=tf.repeat(instance["instrument_idx"][0],(instance["audio"].shape[0]))
    #for key,item in instance.items():
    #    assert(len(item.shape)<2 or item.shape[0]>1)
    return instance

def join_and_window(instance,win_s=4,hop_s=1):
    return window_sample(join_batch(instance),win_s,hop_s)

def rf2cf(row_form):
    return {k:[s[k] for s in row_form] for k in row_form[0].keys()}

# few shot voice cloning


def regularization(batch):
    ir = batch["ir"]
    ir = ir/tf.reduce_max(tf.abs(ir)+1e-10)
    return tf.reduce_mean((ir**2)*tf.cast(tf.linspace(0,1,ir.shape[-1])[None,:],tf.float32))*0.0

# constants
BATCH_SIZE=1

#5e-4 to 1e-5 has worked well

N_DEMO_SAMPLES=int(4*SAMPLE_RATE)

n_fit_windows=int(N_FIT_SECONDS/CLIP_S)

N_FIT_ITERATIONS= 100 if TRAIN_SHARED else int(100*(16/N_FIT_SECONDS)) 

VAL_LR=3e-5 if TRAIN_SHARED else 2e-3
SAVE = True
timestamp=0
DEMO_PATH=f"artefacts/demos/{INSTRUMENT_FAMILY}_{timestamp}_{N_FIT_SECONDS}_{'train_shared' if TRAIN_SHARED else ''}/"

DEMO_IR_DURATION=1.0
DEMO_IR_SAMPLES=int(DEMO_IR_DURATION*SAMPLE_RATE)


val_dataset=list(val_dataset)

# group by instrument id
val_dataset_by_instrument=pydash.collections.group_by(list(val_dataset),lambda x: str(x["instrument"].numpy()))
val_dataset_by_instrument = {k:v for k,v in val_dataset_by_instrument.items() if len(v)>n_fit_windows*2}

# load model
ae_test=model.get_model(SAMPLE_RATE,CLIP_S,FT_FRAME_RATE,Z_SIZE,N_INSTRUMENTS,IR_DURATION,BIDIRECTIONAL,USE_F0_CONFIDENCE,N_HARMONICS,N_NOISE_MAGNITUDES)
# load model weights       

ae_test.set_is_shared_trainable(True)
ae_test.load_weights(checkpoint_path)

ae_test.instrument_weight_metadata["ir"]["initializer"]=lambda batch_size: tf.zeros([batch_size,int(DEMO_IR_DURATION*SAMPLE_RATE)])

if USE_FNR:

    er_samples=int(EARLY_REFLECTION_DURATION*SAMPLE_RATE)

    er_amp=np.ones((er_samples))
    er_amp[er_samples//2:er_samples]=np.linspace(1,0,er_samples//2)

    frame_rate=1000
    n_filter_bands=100
    n_frames=int(frame_rate*DEMO_IR_DURATION)

    ir_fn=ddsp.synths.FilteredNoise(n_samples=DEMO_IR_SAMPLES,
                                       window_size=750,
                                       scale_fn=tf.nn.relu,
                                       initial_bias=0.0001)

    def processing_fn(batched_feature):

        batch_size=batched_feature.shape[0]
        er_ir = tf.nn.tanh(batched_feature[:,:er_samples])

        er_amp=np.ones(DEMO_IR_SAMPLES)
        er_amp[er_samples//2:er_samples]=np.linspace(1,0,er_samples//2)
        er_amp[er_samples:]=0

        er_amp = er_amp[None,:]
        fn_amp= 1-er_amp

        fn_mags=tf.reshape(batched_feature[:,er_samples:],[batch_size,n_frames,n_filter_bands])
        fn_ir=ir_fn(fn_mags)

        ir=fn_ir*fn_amp+tf.pad(er_ir,[[0,0],[0,int(DEMO_IR_DURATION*SAMPLE_RATE)-er_samples]])*er_amp

        #ir = ddsp.core.fft_convolve( fn_ir,er_ir, padding='same', delay_compensation=0)
        return ir

    ae_test.instrument_weight_metadata["ir"]["processing"]=processing_fn

    ae_test.instrument_weight_metadata["ir"]["initializer"]=lambda batch_size: tf.zeros([batch_size,er_samples+n_frames*n_filter_bands])

    ae_test.instrument_weight_metadata["wet_gain"]["initializer"]=lambda batch_size: tf.ones([batch_size,1])*0.5

ae_test.initialize_instrument_weights()
ae_test.set_is_shared_trainable(True)

TMP_CHECKPOINT_PATH="artefacts/tmp_checkpoint"
ae_test.save_weights(TMP_CHECKPOINT_PATH)

for ii,instrument_set in enumerate(list(val_dataset_by_instrument.values())): 

    print(f"instrument nr {ii}")

    ae_test.set_is_shared_trainable(True)
    ae_test.load_weights(TMP_CHECKPOINT_PATH)
    ae_test.initialize_instrument_weights()

    # data
    # reshape data
    fit_data=instrument_set[:n_fit_windows]

    print(f"{len(instrument_set)-5}>={n_fit_windows}")

    assert len(instrument_set)-5>=n_fit_windows

    # Use last 4 windows (16 s) as test data
    test_data=instrument_set[-5:-1]

    def playback_and_save(x,fn):
        print(fn)
        play(x)
        if SAVE:
            os.makedirs(DEMO_PATH,exist_ok=True)
            path=DEMO_PATH+f"recording nr: {ii} "+fn+".wav"
            soundfile.write(path,x,SAMPLE_RATE)

    # convert to column form
    fit_data = rf2cf(fit_data)

    # get one batch for fitting
    fit_batch= next(iter(tf.data.Dataset.from_tensor_slices(fit_data).batch(len(list(fit_data)))))

    playback_and_save(tf.reshape(fit_data["audio"],[-1]),"training data")

    # transform data so that the clips overlap
    fit_batch=join_and_window(fit_batch,4,1)
    fit_data=tf.data.Dataset.from_tensor_slices(fit_batch)
    fit_batched=fit_data.batch(BATCH_SIZE)

    # prepare test data
    test_data = rf2cf(test_data)
    test_batched= tf.data.Dataset.from_tensor_slices(test_data).batch(BATCH_SIZE)
    

    fit_losses=[]
    tst_losses=[]

    # set up optimizer
    val_optimizer = tf.keras.optimizers.Adam(learning_rate=VAL_LR)

    for i in tqdm.tqdm(range(N_FIT_ITERATIONS)):
        fit_batched=fit_batched.shuffle(100)

        epoch_loss=0
        batch_counter=0
        test_epoch_loss=0
        test_batch_counter=0

        for fit_batch in fit_batched:
            with tf.GradientTape() as tape:

              output = ae_test(fit_batch,train_shared=TRAIN_SHARED)
              loss_value=spectral_loss(fit_batch["audio"],output["audio_synth"])+regularization(output)
              epoch_loss+=loss_value.numpy()
              batch_counter+=1
              gradients = tape.gradient(loss_value, ae_test.trainable_weights)
            val_optimizer.apply_gradients(zip(gradients, ae_test.trainable_weights))
        fit_losses.append(epoch_loss/batch_counter)


        for test_batch in test_batched:
            test_output=ae_test(test_batch,train_shared=False)
            loss_value=spectral_loss(test_batch["audio"],test_output["audio_synth"])   
            test_epoch_loss+=loss_value.numpy()
            test_batch_counter+=1
        tst_losses.append(test_epoch_loss/test_batch_counter)

        if i%50==0:

            print("target")        
            play(tf.reshape(fit_batch["audio"],(-1)))

            print("estimate")     
            play(tf.reshape(output['audio_synth'],(-1)))
            # loss plot
            plt.plot(tst_losses,label="tst")
            plt.plot(fit_losses,label="trn")
            plt.yscale("log")
            plt.legend()
            plt.show()

            ir=output['ir'][0]

            plt.plot(ir)
            plt.show()

            play(tf.reshape(ir,(-1)))

            plt.imshow(ddsp.spectral_ops.compute_mel(ir).numpy().T,aspect="auto",origin="lower")
            plt.show()

            print(f"wet gain: {output['wet_gain']['controls']['gain_scaled']}")
            print(f"dry gain: {output['dry_gain']['controls']['gain_scaled']}")

    plt.plot(tst_losses,label="tst")
    plt.plot(fit_losses,label="trn")
    plt.yscale("log")
    plt.legend()
    plt.show()

    print(">> seen data:")
    playback_and_save(tf.reshape(fit_batch["audio"],[-1]),"training target")

    playback_and_save(tf.reshape(output["audio_synth"],[-1]),"training estimate")

    transposed_fit_batch = copy.deepcopy(fit_batch)
    transposed_fit_batch["f0_hz"]=transposed_fit_batch["f0_hz"]*0.7
    transposed_output=ae_test(transposed_fit_batch,train_shared=False)
    playback_and_save(tf.reshape(transposed_output['audio_synth'],(-1)),"transposed down")

    transposed_fit_batch = copy.deepcopy(fit_batch)
    transposed_fit_batch["f0_hz"]=transposed_fit_batch["f0_hz"]*1.3
    transposed_output=ae_test(transposed_fit_batch,train_shared=False)
    playback_and_save(tf.reshape(transposed_output['audio_synth'],(-1)),"transposed up")

    transposed_fit_batch = copy.deepcopy(fit_batch)
    transposed_fit_batch["loudness_db"]=transposed_fit_batch["loudness_db"]-12
    transposed_output=ae_test(transposed_fit_batch,train_shared=False)
    playback_and_save(tf.reshape(transposed_output['audio_synth'],(-1)),"loudness -12 db")

    transposed_fit_batch = copy.deepcopy(fit_batch)
    transposed_fit_batch["loudness_db"]=transposed_fit_batch["loudness_db"]-6
    transposed_output=ae_test(transposed_fit_batch,train_shared=False)
    playback_and_save(tf.reshape(transposed_output['audio_synth'],(-1)),"loudness -6 db")

    transposed_fit_batch = copy.deepcopy(fit_batch)
    transposed_fit_batch["loudness_db"]=transposed_fit_batch["loudness_db"]+6
    transposed_output=ae_test(transposed_fit_batch,train_shared=False)
    playback_and_save(tf.reshape(transposed_output['audio_synth'],(-1)),"loudness +6 db")

    transposed_fit_batch = copy.deepcopy(fit_batch)
    transposed_fit_batch["loudness_db"]=transposed_fit_batch["loudness_db"]+12
    transposed_output=ae_test(transposed_fit_batch,train_shared=False)
    playback_and_save(tf.reshape(transposed_output['audio_synth'],(-1)),"loudness +12 db")

    transposed_fit_batch = copy.deepcopy(fit_batch)
    transposed_fit_batch["f0_confidence"]=transposed_fit_batch["f0_confidence"]*0.9
    transposed_output=ae_test(transposed_fit_batch,train_shared=False)
    playback_and_save(tf.reshape(transposed_output['audio_synth'],(-1)),"low confidence")

    transposed_fit_batch = copy.deepcopy(fit_batch)
    transposed_fit_batch["f0_confidence"]=transposed_fit_batch["f0_confidence"]*0.5
    transposed_output=ae_test(transposed_fit_batch,train_shared=False)
    playback_and_save(tf.reshape(transposed_output['audio_synth'],(-1)),"very low f0 confidence")

    transposed_fit_batch = copy.deepcopy(fit_batch)
    transposed_fit_batch["f0_confidence"]=transposed_fit_batch["f0_confidence"]*0.0
    transposed_output=ae_test(transposed_fit_batch,train_shared=False)
    playback_and_save(tf.reshape(transposed_output['audio_synth'],(-1)),"no f0 confidence")

    print(">> unseen data:")
    playback_and_save(tf.reshape(test_batch["audio"][:,:N_DEMO_SAMPLES],(-1)),"unseen target")

    test_batch_output = ae_test(test_batch,train_shared=False)
    playback_and_save(tf.reshape(test_batch_output['audio_synth'][:,:N_DEMO_SAMPLES],(-1)),"unseen estimate")


